# Implémentation de Coordinate Descent
---
**Aymen.B**

Je me repose sur l'article *"Coordinate Descent Algorithms"* de **Stephen J. Wright**

## Formulation du problème

On cherche à résoudre un problème de cette forme :

$$\min_xh(x):=f(x)+\lambda\Omega(x),$$

- $f:\mathbb{R}^n\to\mathbb{R}$ doit être continue lisse.
- $\Omega:\mathbb{R}^n\to\mathbb{R}$ est la fonction de pénalisation, non lisse mais séparable.
  - $\Omega(x)=\sum_{i=1}^n\Omega_i(x_i)$ avec $\Omega_i:\mathbb{R}\to\mathbb{R}$ pour tout $i$.
- $\lambda > 0$

## CD (Coordinate Descent)

### Théorie

L'idée est de mettre à jour une seule coordonnée $x_i$ à la fois. À chaque itération $k$, on choisit un indice $i_k \in [{1, \ldots, n}]$, et on résout le problème unidimensionnel :

$$x_{i_k}^{k+1}=\arg\min_{\chi\in\mathbb{R}}\left\{f(x^k)+\nabla f(x^k)_i(\chi-x_i^k)+\frac{1}{2\alpha_k}(\chi-x_i^k)^2+\lambda\Omega_i(\chi)\right\}.$$

Ceci correspond à un développement de Taylor à l’ordre 1 de $f$ autour de $x^k$ en la direction $i$, régularisé par un terme quadratique. On utilise ici une condition de Lipschitzianité du gradient selon $x_i$. Ce qui fais intervenir une constante $L_i$ telle que:

$$|\nabla f(x+te_i)_i-\nabla f(x)_i|\leq L_i|t|$$

On note $\alpha_k = 1 / L_i$

On obtiens donc la mise à jour suivante :

$$x^{k+1} = x^k + (z^k_{i_k} - x^k_{i_k})e_{i_k}$$

avec

$$z^k_{i_k} := \arg\min_{\chi} \left\{ \nabla f(x^k)_{i_k}(\chi - x^k_{i_k}) + \frac{1}{2\alpha_k}(\chi - x^k_{i_k})^2 + \lambda\Omega_{i_k}(\chi) \right\}$$

Or on remarque que :

$$z^k_{i_k} = \text{prox}_{\lambda\alpha_k\Omega_{i_k}}\left(x^k_{i_k} - \alpha_k \nabla f(x^k)_{i_k}\right)$$

avec 

$$\mathrm{prox}_{\gamma g}(z):=\arg\min_{x\in\mathbb{R}^n}\left\{\frac{1}{2\gamma}\|x-z\|^2+g(x)\right\}$$


### Algorithme

$$ \boxed{ \begin{array}{ll} 
\textbf{Coordinate Descent Algorithm} \\
\\
\textbf{Step 0.} & \text{Take } x^0 \in \mathbb{R}^n \text{ and set } k \leftarrow 0. \\
\\
\textbf{Step k.} & \text{Repeat until convergence:} \\
& \quad \text{Choose index } i_k \in \{1, 2, \ldots, n\} \\
& \quad z^k_{i_k} \leftarrow \text{prox}_{\lambda\alpha_k\Omega_{i_k}}\left(x^k_{i_k} - \alpha_k \nabla f(x^k)_{i_k}\right) \\
& \quad x^{k+1} \leftarrow x^k + (z^k_{i_k} - x^k_{i_k})e_{i_k} \\
& \quad k \leftarrow k + 1
\end{array} } $$

In [10]:
from linear_tools import cd

> **Note :** On as remarqué que l'algorithme ci-dessus ne converger pas correctement lorsque le test de converge avait lieu à chaque itération sur $i_k$. L'algorithme converge correctement lorsque le test à lieu après un cycle d'optimisation où tout les $i_k$ on pu être optimisé. 

## Test de comparaison

In [11]:
import numpy as np

import sys, pathlib
# You are in .../CAS LINEAIRE/OUTILS/.
project_root = pathlib.Path.cwd().parent.parent.parent.parent        # goes up to .../CAS LINEAIRE
sys.path.append(str(project_root))              # makes comp_translate visible

from comp_translate.SaveLoadTxt import dump_txt, load_txt

In [12]:
project_root

WindowsPath('c:/Users/Le R/Desktop/Code/Projets/Geneve/STAGE-GENEVE')

### Setup

In [13]:
n, p = 100, 50
sigma = 0.1
λ = 0.1
X, y = load_txt(project_root / "comp_translate/data/ISTA.txt")
y = y.reshape(-1, 1)

### Results

In [19]:
from numpy.linalg import norm

l_L = [norm(X[:, j])**2 for j in range(p)]

def f(b: np.ndarray) -> float:
    """0.5‖y – Xb‖²"""
    r = y - X @ b
    return 0.5 * np.linalg.norm(r)**2

def grad_f(b: np.ndarray) -> np.ndarray:
    """∇f(b) = –Xᵀ(y – Xb)"""
    return -X.T @ (y - X @ b)

def prox_l1(z: np.ndarray, step) -> np.ndarray:
    """Soft–thresholding (prox for λ‖·‖₁)."""
    return np.sign(z) * np.maximum(np.abs(z) - step, 0.0)

beta0 = np.zeros((p,1))

In [20]:
beta_hat = cd(grad_f=grad_f,prox_O=prox_l1,x0=beta0, L=l_L,lmbda=λ, max_iter=10000, tol=1e-9)

In [21]:
f(beta_hat) + λ * np.linalg.norm(beta_hat, 1) 

np.float64(4.279514355446482)

In [22]:
beta_hat

array([[-0.22167364],
       [-0.81076458],
       [ 1.21973272],
       [ 2.12965552],
       [ 0.85821933],
       [-0.31706253],
       [ 0.6494031 ],
       [-0.27917011],
       [ 1.1051304 ],
       [-0.11074081],
       [ 1.3709152 ],
       [-0.22553451],
       [-0.460652  ],
       [ 0.27874045],
       [-0.06087302],
       [-0.41398735],
       [ 2.03298944],
       [ 0.79580748],
       [ 0.63172314],
       [ 0.58165937],
       [ 0.10317614],
       [ 0.26391064],
       [ 0.28993509],
       [-0.90545068],
       [ 1.90577069],
       [-0.67079577],
       [ 0.93406906],
       [ 0.67438592],
       [ 0.89318611],
       [-0.60058256],
       [-1.99923067],
       [-0.53729802],
       [-1.04430345],
       [-0.26719144],
       [ 0.45900136],
       [-0.14062488],
       [ 2.33126363],
       [ 0.66337138],
       [ 1.96720417],
       [-0.63909014],
       [ 1.26966516],
       [ 0.26028179],
       [ 2.00004166],
       [-0.2743964 ],
       [-1.11617992],
       [-1